# Source

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

import seaborn as sns

from pymongo import MongoClient
mc = MongoClient(os.getenv('mongo_host'),
                    username=os.getenv('mongo_user'),
                    password=os.getenv('mongo_pass'),
                    authSource=os.getenv('mongo_db_auth'),
                    authMechanism='SCRAM-SHA-256')

db = mc['bfv_processing']

In [2]:
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

pio.templates.default = "none"

# Seaborn Config
p1 = sns.color_palette("dark:#0B6872", n_colors=10)
p2 = sns.color_palette("dark:#00D3BC", n_colors=10)
p3 = sns.color_palette("light:#040404", n_colors=10)
p4 = sns.color_palette("dark:#F4E9C3", n_colors=10)
p5 = sns.color_palette("dark:#EF8B79", n_colors=10)

# Colors
gumi_palette = ['#0B6872','#00D3BC','#040404','#F4E9C3','#EF8B79']

gumi_layout_update_dict = dict(
    font_family="sans-serif",
    font_color="grey",
    title_font_family="sans-serif",
    title_font_size=20,
    title_font_color="black",
    legend_title_font_color="grey",
    showlegend=False,
    paper_bgcolor='#E9F2F1',
    plot_bgcolor='rgba(0,0,0,0)'
)

# Helpers

In [11]:
def gen_ranked_players(metric, q={}):

    mp = pd.DataFrame(list(db.processed_match_players.find(q, {'_id':1, metric:1, 'match_id':1, 'team_orientation':1, 'team_status':1, 'map':1, 'mode':1})))

    mp["finisher_rank"] = np.nan
    mp.loc[mp[mp.team_status != 'dnf'].index, 'finisher_rank'] = mp.groupby("match_id")[metric].rank("dense", ascending=False).astype(int)
    mp['overall_rank'] = mp.groupby("match_id")[metric].rank("dense", ascending=False).astype(int)
    mp['team_rank'] = mp.groupby(["match_id", 'team_status'])[metric].rank("dense", ascending=False).astype(int)
    
    out = mp

    return out

In [12]:
map_filter = "Operation Underground" # As it appears on TN
mode_filter = "Breakthrough" # As it Appears on TN
gen_ranked_players('score', {"map":map_filter, "mode":mode_filter})

,_id,match_id,map,mode,team_status,score,team_orientation,finisher_rank,overall_rank,team_rank
0,psn_1409210561723200320_psn_nitobetico93,psn_1409210561723200320,Operation Underground,Breakthrough,won,6478,attacker,4.0,4,1
1,psn_1409210561723200320_psn_brusk_sasan_44,psn_1409210561723200320,Operation Underground,Breakthrough,won,6350,attacker,5.0,5,2
2,psn_1409210561723200320_psn_jurriman035,psn_1409210561723200320,Operation Underground,Breakthrough,won,6249,attacker,6.0,6,3
3,psn_1409210561723200320_psn_finstan,psn_1409210561723200320,Operation Underground,Breakthrough,won,6068,attacker,7.0,7,4
4,psn_1409210561723200320_psn_THFCBOY26,psn_1409210561723200320,Operation Underground,Breakthrough,won,5879,attacker,9.0,9,5
...,...,...,...,...,...,...,...,...,...,...
33830,psn_1453857961781108800_psn_Kultavader69,psn_1453857961781108800,Operation Underground,Breakthrough,dnf,0,unknown,NaN,77,6
33831,psn_1453857961781108800_psn_wheemzy,psn_1453857961781108800,Operation Underground,Breakthrough,dnf,0,unknown,NaN,77,6
33832,psn_1453857961781108800_psn_n64_R2D2,psn_1453857961781108800,Operation Underground,Breakthrough,dnf,0,unknown,NaN,77,6
33833,psn_1453857961781108800_psn_hypebrenes,psn_1453857961781108800,Operation Underground,Breakthrough,dnf,0,unknown,NaN,77,6


# Match Player Reference Table
For a match, a player has there projected performance going in. 

1. ```Player 1 - KPM 1.05 | Player 3 - KPM 1.05```
1. ```Player 2 - KPM 1.10 | Player 4 - KPM 1.00```

Team 1 is thus expected to have 1.15 KPM, Team 1 is 1.05 KPM. Thus, Team 1 has a matchup advantage of .10 KPM. If the resulting difference was higher than .10, the amount above .10 would be the performance difference in teams.

1. Expected 1.15 | 1.05 
1. Observed 1.10 | 1.07 (.03 on Team 1)

Team 1 

# Operation Underground - POC

## Overview

In [13]:
def summary(match_info):
    """
    Prints high-level Summary for data
    """
    
    # Check we have the right data
    print(f"Unique Maps: {match_info['map'].unique()}")
    print(f"Unique Modes: {match_info['mode'].unique()}\n")
    
    # How Many Players
    num_unique_players = len(match_info['player_id'].unique())
    num_unique_matches = len(match_info['match_id'].unique())
    num_match_players = len(match_info)
    print(f"Unique Players: {num_unique_players}")
    print(f"Unique Matches: {num_unique_matches}")
    print(f"Repeated Players per Match: {(num_match_players - num_unique_players) / num_unique_matches}")

In [14]:
def match_metric_summary(matches, metric_name, plot=True):
    """
    Computes a summary of a particular metric
    """
    
    if metric_name not in matches.columns:
        raise ValueError(f"Metric {metric_name} not in Data. Try one of {matches.columns}")

    if plot:
        px.histogram(matches.reset_index(),
                     x=metric_name, 
                     color='team_status',
                     marginal='rug',
                     barmode='overlay').show()
    
    result = matches.pivot_table(index='match_id', columns='team_status', values=metric_name, aggfunc='first').mean(axis=0)
    print(f"Winner to Loser Avg. Difference: {result['won'] - result['lost']}")

In [3]:
# Read in the match_players, Dataframe containing all matches the performance of the players in them
# Ideally this is done for 1 map and mode, both of which have filters below
# This cell outputs a sampling view of the data
map_filter = "Operation Underground" # As it appears on TN
mode_filter = "Breakthrough" # As it Appears on TN
match_info = pd.DataFrame(list(db.processed_match_players.find({"map":map_filter, "mode":mode_filter}, {"_id":0})))
match_info.sample(50).head(50)

,match_id,map,mode,game_duration_m,team,team_status,player_id,kills,deaths,kills_per_death,...,team_rank,match_start_time,team_orientation,player_time,AER,aggression_rating,efficiency_rating,adj_kpm,adj_spm,duration_m
4498,psn_1453215914464666688,Operation Underground,Breakthrough,41m 13s,Unknown,dnf,psn_DarkSteelShine-D,0,0,0.00,...,7,2021-10-26 23:05:00,unknown,1.000000,NaN,NaN,NaN,0.000000,6.065507,41.216667
18012,psn_1440502502877263424,Operation Underground,Breakthrough,28m 03s,GER,won,psn_Youawetgoose,36,12,3.00,...,3,2021-09-21 20:53:00,defender,23.999861,8.175021,3.633354,4.541667,1.283422,493.404635,28.050000
12661,psn_1440744648417128512,Operation Underground,Breakthrough,38m 50s,GER,won,psn_HUNTERTHY,18,15,1.20,...,24,2021-09-22 13:06:00,defender,32.999371,3.480482,1.465482,2.015000,0.463519,202.712446,38.833333
18774,psn_1439706030103374528,Operation Underground,Breakthrough,32m 54s,UK,lost,psn_POTRIOTIC----COM,13,9,1.44,...,29,2021-09-19 16:13:00,attacker,30.000599,3.241590,1.051312,2.190278,0.395137,152.218845,32.900000
33975,psn_1524575064685277760,Operation Underground,Breakthrough,39m 13s,UK,lost,psn_ItsMrRogersHood,0,2,0.00,...,38,2022-05-11 20:59:00,attacker,9.000185,1.031488,0.270550,0.760938,0.000000,12.418190,39.216667
15875,psn_1405893257659665088,Operation Underground,Breakthrough,33m 17s,UK,won,psn_EmperorOfUnicorn,66,17,3.88,...,1,2021-06-18 08:54:00,attacker,28.999925,9.225846,4.465184,4.760662,1.982974,579.809715,33.283333
26422,psn_1451931860977180096,Operation Underground,Breakthrough,36m 07s,UK,lost,psn_tree-magnet,34,18,1.89,...,5,2021-10-23 09:58:00,attacker,33.000336,5.261818,2.452096,2.809722,0.941394,353.964006,36.116667
2138,psn_1451951029470813824,Operation Underground,Breakthrough,15m 02s,UK,won,psn_the_chubbydoor,5,12,0.42,...,12,2021-10-23 11:36:00,attacker,11.999699,1.418511,0.872939,0.545573,0.332594,106.097561,15.033333
9872,psn_1458626865972229440,Operation Underground,Breakthrough,29m 54s,GER,won,psn_opencast-precis3,1,2,0.50,...,33,2021-11-10 20:15:00,defender,4.000000,1.631250,0.725000,0.906250,0.033445,16.053512,29.900000
14816,psn_1439318728831597248,Operation Underground,Breakthrough,21m 34s,GER,lost,psn_katanablade2001,0,4,0.00,...,30,2021-09-18 14:23:00,defender,5.000000,1.654781,0.929000,0.725781,0.000000,43.075734,21.566667


In [4]:
summary(match_info)

NameError: name 'summary' is not defined

## Win/loss Summary

In [143]:
def metric_win_rate_plot(matches, metric, num_bins):
    
    temp = matches.copy()
    even_increment = (temp[metric].quantile(.9) - temp[metric].quantile(.1)) / num_bins
    temp[f'{metric}_simple'] = (temp[metric] / even_increment).round() * even_increment
    plot_df = temp.groupby(f'{metric}_simple', as_index=False).agg({'a_win':'mean'})
    
    figure = px.area(plot_df.sort_values(f'{metric}_simple'), 
                     x=f'{metric}_simple', 
                     y='a_win')
    figure.update_yaxes(range=[0, .8])
    figure.update_xaxes(range=[temp[metric].quantile(.1), temp[metric].quantile(.9)])
    figure.update_layout(**gumi_layout_update_dict)
    figure.show()

In [144]:
def win_loss_summary(match_info, num_bins=50):
    
    matches = (
        match_info
        .groupby(['match_id', 'team_status', 'team_orientation'], as_index=False)
        .agg({
                'match_id':'nunique',
                'kills':'sum',
                'score':'sum',
                'deaths':'sum',
                'kills_per_min':'mean',
                'adj_spm':'mean'
            })
    )
    
    a_win_rate = (len(matches[(matches.team_orientation=='attacker') & (matches.team_status=='won')]) / 
        len(matches[(matches.team_orientation=='attacker')])
    )
    d_win_rate = (
        len(matches[(matches.team_orientation=='defender') & (matches.team_status=='won')]) / 
        len(matches[(matches.team_orientation=='defender')])
    )
    
    print(f"Attacker Win Rate: {round(a_win_rate*100)}%")
    print(f"Defender Win Rate: {round(d_win_rate*100)}%")
    
    matches['a_win'] = 0
    matches.loc[matches[(matches.team_orientation=='attacker') & (matches.team_status=='won')].index, 'a_win'] = 1
    matches.loc[matches[(matches.team_orientation=='defender') & (matches.team_status=='lost')].index, 'a_win'] = 0
    
    for metric in ['kills', 'score', 'kills_per_min', 'adj_spm']:
        print(f"\n{metric}\n")
        wl = matches.pivot_table(index='team_orientation', columns='team_status', values=metric, aggfunc='mean')
        print(wl)
        
        metric_win_rate_plot(matches, metric, num_bins)

In [145]:
win_loss_summary(match_info, 25)

Attacker Win Rate: 52%
Defender Win Rate: 48%

kills

team_status             dnf        lost         won
team_orientation                                   
attacker                NaN  741.369565  448.722772
defender                NaN  425.138614  842.267760
unknown           28.349741         NaN    0.000000



score

team_status                dnf           lost            won
team_orientation                                            
attacker                   NaN  278267.021739  187507.490099
defender                   NaN  167281.069307  347255.284153
unknown           11764.484456            NaN       0.000000



kills_per_min

team_status           dnf      lost       won
team_orientation                             
attacker              NaN  0.839563  0.859138
defender              NaN  0.860316  0.857725
unknown           0.27899       NaN  0.000000



adj_spm

team_status             dnf        lost         won
team_orientation                                   
attacker                NaN  197.718337  252.468716
defender                NaN  214.051143  239.194465
unknown           33.518038         NaN    0.000000


## Disparity

In [216]:
def disparity(match_info, metric):
    
    temp = match_info[match_info.team_orientation != 'unknown']
    temp['team_label'] = match_info.team_status + " - " + match_info.team_orientation
    temp['player_rank'] = 0
    
    for match_id in temp.match_id.unique():
        filtered = temp[temp.match_id == match_id]
        
        temp.loc[filtered[filtered.team_status=='won'].index, 'player_rank'] = filtered[filtered.team_status=='won'][metric].rank(method='first', ascending=False)
        temp.loc[filtered[filtered.team_status=='lost'].index, 'player_rank'] = filtered[filtered.team_status=='lost'][metric].rank(method='first', ascending=False)

    result = temp.pivot_table(index='player_rank', columns='team_orientation', values=metric, aggfunc='mean')
    print(f"A/D Disparity Overall {(result['attacker']-result['defender']).mean()}")
        
        
    result = temp.pivot_table(index='player_rank', columns='team_label', values=metric, aggfunc='mean')
    print(f"A/D Disparity (W/L) {(result['won - attacker']-result['lost - defender']).mean()}")
    print(f"D/A Disparity (W/L) {(result['won - defender']-result['lost - attacker']).mean()}")
    
    return temp.pivot_table(index='player_rank', columns='team_status', values=metric, aggfunc='mean')

In [217]:
maps = [
    "MP_WakeIsland",
    "Operation Underground",
    "Iwo Jima",
    "Aerodrome",
    "Solomon Islands",
    "Provence",
    "Rotterdam",
    "Devastation"
]

for map_filter in maps:
    info = pd.DataFrame(list(db.processed_match_players.find({"map":map_filter, "mode":mode_filter}, {"_id":0})))
    print(map_filter)
    disparity(info, 'adj_spm')

MP_WakeIsland


C:\Users\atawz\AppData\Local\Temp/ipykernel_22596/4231476037.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\atawz\AppData\Local\Temp/ipykernel_22596/4231476037.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



A/D Disparity Overall 10.012080920352732
A/D Disparity (W/L) 37.48075576230246
D/A Disparity (W/L) 1.7385978576574659
Operation Underground
A/D Disparity Overall -1.8581738989465686
A/D Disparity (W/L) 33.4420473835204
D/A Disparity (W/L) 36.905059595519
Iwo Jima
A/D Disparity Overall 4.982799376125227
A/D Disparity (W/L) 37.298104432672346
D/A Disparity (W/L) 1.7192088895462323
Aerodrome
A/D Disparity Overall 27.432745553445393
A/D Disparity (W/L) 41.00614984458972
D/A Disparity (W/L) -3.833022562945394
Solomon Islands
A/D Disparity Overall 10.582248414248086
A/D Disparity (W/L) 37.97683888920132
D/A Disparity (W/L) 3.1817343103665197
Provence
A/D Disparity Overall -14.417506072957641
A/D Disparity (W/L) 28.937435609580007
D/A Disparity (W/L) 32.68624795415714
Rotterdam
A/D Disparity Overall 3.5755699329174577
A/D Disparity (W/L) 26.974766331259374
D/A Disparity (W/L) 10.921986346399805
Devastation
A/D Disparity Overall 18.134518322475653
A/D Disparity (W/L) 57.72458040367531
D/A Disp

In [220]:
px.histogram(match_info[(match_info.team_orientation=='attacker') & (match_info.team_status=='lost')].groupby('match_id').agg({'deaths':'sum'}))

In [133]:
map_summary(match_info)

Metric: kills
Winner to Loser Avg. Difference: 58.02356020942409
Metric: duration_m
Winner to Loser Avg. Difference: 0.0
Metric: deaths
Winner to Loser Avg. Difference: -29.0130890052356
Metric: kills_per_death
Winner to Loser Avg. Difference: 0.3100633667936794


kills  duration_m  deaths  \
match_id                team_status                              
psn_1185002410657891072 dnf              2   27.050000       6   
                        lost           573   27.050000     411   
                        won            651   27.050000     487   
psn_1185009344618349312 dnf              0   30.500000       0   
                        lost            23   30.500000      29   
...                                    ...         ...     ...   
psn_1462645947334572096 lost           205   12.366667     199   
                        won            250   12.366667     137   
psn_1462853257382032960 dnf             39   29.183333      32   
                        lost           662   29.183333     545   
                        won            487   29.183333     361   

                                     kills_per_death  
match_id                team_status                   
psn_1185002410657891072 dnf                 0.333333  
                        lost                1.394161  
                        won                 1.336756  
psn_1185009344618349312 dnf                      NaN  
                        lost                0.793103  
...                                              ...  
psn_1462645947334572096 lost                1.030151  
                        won                 1.824818  
psn_1462853257382032960 dnf                 1.218750  
                        lost                1.214679  
                        won                 1.349030  

[1146 rows x 4 columns]